
# Assignment III: CUDA Basics II
## [DD2360] Applied GPU Programming
Group 8

Valeria Grotto (vgrotto@kth.se) | Martin Forslund (uz6@kth.se)

***

## Exercise 2

In [39]:
# check GPU architecture
!nvidia-smi

Wed Dec  6 13:49:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [40]:
# Check CPU information
!lscpu

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         46 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  2
  On-line CPU(s) list:   0,1
Vendor ID:               GenuineIntel
  Model name:            Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:          6
    Model:               79
    Thread(s) per core:  2
    Core(s) per socket:  1
    Socket(s):           1
    Stepping:            0
    BogoMIPS:            4399.99
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clf
                         lush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_
                         good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fm
                         a cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hyp
                         ervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd i

In [41]:
! git clone https://github.com/KTH-HPC/sputniPIC-DD2360.git

Cloning into 'sputniPIC-DD2360'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 78 (delta 0), reused 6 (delta 0), pack-reused 68
Receiving objects: 100% (78/78), 52.13 KiB | 539.00 KiB/s, done.
Resolving deltas: 100% (18/18), done.


Compile and run the current version of the code, which still runs on CPU, and get familiar with the particle mover_PC() function in Particles.cu. Some background information about the code and how to run a simple simulation can be found in Introduction of spunitGPU.pdf Download Introduction of spunitGPU.pdf. For this exercise, use the GEM_2D.inp input file.

In [42]:
# modify makefile
with open('sputniPIC-DD2360/Makefile', 'r') as file:
    content = file.read()

# Modify GPU architecture
content = content.replace('sm_30', 'sm_75')

# Write modified content back to the Makefile
with open('sputniPIC-DD2360/Makefile', 'w') as makefile:
    makefile.write(content)

In [43]:
content

"# From https://x.momo86.net/?p=29\n\nCXX=g++\nCXXFLAGS=-std=c++11 -I./include -O3 -g -Xcompiler -Wall\n\nNVCC=nvcc\nARCH=sm_75\nNVCCFLAGS= -I./include -arch=$(ARCH) -std=c++11 -O3 -g -Xcompiler -Wall --compiler-bindir=$(CXX)\n\nSRCDIR=src\nSRCS=$(shell find $(SRCDIR) -name '*.cu' -o -name '*.cpp')\n\nOBJDIR=src\nOBJS=$(subst $(SRCDIR),$(OBJDIR), $(SRCS))\nOBJS:=$(subst .cpp,.o,$(OBJS))\nOBJS:=$(subst .cu,.o,$(OBJS))\n\nBIN := ./bin\nTARGET=sputniPIC.out\n\nall: dir $(BIN)/$(TARGET)\n\ndir: ${BIN}\n  \n${BIN}:\n\tmkdir -p $(BIN)\n\n$(BIN)/$(TARGET): $(OBJS)\n\t$(NVCC) $(NVCCFLAGS) $+ -o $@\n\n$(SRCDIR)/%.o: $(SRCDIR)/%.cu\n\t$(NVCC) $(NVCCFLAGS) $< -c -o $@\n\n$(OBJDIR)/%.o: $(SRCDIR)/%.cpp\n\t[ -d $(OBJDIR) ] || mkdir $(OBJDIR)\n\t$(NVCC) $(CXXFLAGS) $< -c -o $@\n\nclean:\n\trm -rf $(OBJS)\n\trm -rf $(TARGET)\n"

In [44]:
%cd sputniPIC-DD2360
! make

/content/sputniPIC-DD2360/sputniPIC-DD2360/sputniPIC-DD2360
mkdir -p ./bin
[ -d src ] || mkdir src
nvcc -std=c++11 -I./include -O3 -g -Xcompiler -Wall src/TransArraySpace3D.cpp -c -o src/TransArraySpace3D.o
[ -d src ] || mkdir src
nvcc -std=c++11 -I./include -O3 -g -Xcompiler -Wall src/Grid.cpp -c -o src/Grid.o
[ -d src ] || mkdir src
nvcc -std=c++11 -I./include -O3 -g -Xcompiler -Wall src/InterpDensSpecies.cpp -c -o src/InterpDensSpecies.o
[ -d src ] || mkdir src
nvcc -std=c++11 -I./include -O3 -g -Xcompiler -Wall src/sputniPIC.cpp -c -o src/sputniPIC.o
In file included from src/sputniPIC.cpp:25:
./include/IC.h: In function ‘void initGEM(parameters*, grid*, EMfield*, EMfield_aux*, particles*, interpDensSpecies*)’:
./include/IC.h:82:25: warning: unused variable ‘sign’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-variable-Wunused-variable]8;;]
   82 |     double prob, theta, sign;
      |                         ^~~~
[ -d src ] || mkdir src
nvcc -std=c+

In [45]:
# create data folder to place the output
! mkdir data

In [46]:
# run the code without modivications
! ./bin/sputniPIC.out inputfiles/GEM_2D.inp


-------------------------
sputniPIC Sim. Parameters
-------------------------
Number of species    = 4
Number of particles of species 0 = 4096000	 (MAX = 4096000)  QOM = -64
Number of particles of species 1 = 4096000	 (MAX = 4096000)  QOM = 1
Number of particles of species 2 = 4096000	 (MAX = 4096000)  QOM = -64
Number of particles of species 3 = 4096000	 (MAX = 4096000)  QOM = 1
x-Length                 = 40
y-Length                 = 20
z-Length                 = 1
Number of cells (x)      = 256
Number of cells (y)      = 128
Number of cells (z)      = 1
Time step                = 0.25
Number of cycles         = 10
Results saved in: data
*************************************************
**  Initialize GEM Challenge with Pertubation  **
*************************************************
** B0x = 0.0195
** B0y = 0
** B0z = 0
** Delta (current sheet thickness) = 0.5
** rho species 0 = 1 CURRENT SHEET 
** rho species 1 = 1 CURRENT SHEET 
** rho species 2 = 0.02 BACKGROUND 
** rho species

## GPU Optimized version

In [47]:
%%writefile src/sputniPIC.cpp

/** A mixed-precision implicit Particle-in-Cell simulator for heterogeneous systems **/

// Allocator for 2D, 3D and 4D array: chain of pointers
#include "Alloc.h"

// Precision: fix precision for different quantities
#include "PrecisionTypes.h"
// Simulation Parameter - structure
#include "Parameters.h"
// Grid structure
#include "Grid.h"
// Interpolated Quantities Structures
#include "InterpDensSpecies.h"
#include "InterpDensNet.h"

// Field structure
#include "EMfield.h" // Just E and Bn
#include "EMfield_aux.h" // Bc, Phi, Eth, D

// Particles structure
#include "Particles.h"
#include "Particles_aux.h" // Needed only if dointerpolation on GPU - avoid reduction on GPU

// Initial Condition
#include "IC.h"
// Boundary Conditions
#include "BC.h"
// timing
#include "Timing.h"
// Read and output operations
#include "RW_IO.h"


int main(int argc, char **argv){

    // Read the inputfile and fill the param structure
    parameters param;
    // Read the input file name from command line
    readInputFile(&param,argc,argv);
    printParameters(&param);
    saveParameters(&param);

    // Timing variables
    double iStart = cpuSecond();
    double iMover, iInterp, eMover = 0.0, eInterp= 0.0;

    // Set-up the grid information
    grid grd;
    setGrid(&param, &grd);

    // Allocate Fields
    EMfield field;
    field_allocate(&grd,&field);
    EMfield_aux field_aux;
    field_aux_allocate(&grd,&field_aux);


    // Allocate Interpolated Quantities
    // per species
    interpDensSpecies *ids = new interpDensSpecies[param.ns];
    for (int is=0; is < param.ns; is++)
        interp_dens_species_allocate(&grd,&ids[is],is);
    // Net densities
    interpDensNet idn;
    interp_dens_net_allocate(&grd,&idn);

    // Allocate Particles
    particles *part = new particles[param.ns];
    // allocation
    for (int is=0; is < param.ns; is++){
        particle_allocate(&param,&part[is],is);
    }

    // Initialization
    initGEM(&param,&grd,&field,&field_aux,part,ids);


    // **********************************************************//
    // **** Start the Simulation!  Cycle index start from 1  *** //
    // **********************************************************//
    for (int cycle = param.first_cycle_n; cycle < (param.first_cycle_n + param.ncycles); cycle++) {

        std::cout << std::endl;
        std::cout << "***********************" << std::endl;
        std::cout << "   cycle = " << cycle << std::endl;
        std::cout << "***********************" << std::endl;

        // set to zero the densities - needed for interpolation
        setZeroDensities(&idn,ids,&grd,param.ns);



        // implicit mover
        iMover = cpuSecond(); // start timer for mover
        for (int is=0; is < param.ns; is++)
            mover_PC_gpu(&part[is],&field,&grd,&param);
        eMover += (cpuSecond() - iMover); // stop timer for mover




        // interpolation particle to grid
        iInterp = cpuSecond(); // start timer for the interpolation step
        // interpolate species
        for (int is=0; is < param.ns; is++)
            interpP2G(&part[is],&ids[is],&grd);
        // apply BC to interpolated densities
        for (int is=0; is < param.ns; is++)
            applyBCids(&ids[is],&grd,&param);
        // sum over species
        sumOverSpecies(&idn,ids,&grd,param.ns);
        // interpolate charge density from center to node
        applyBCscalarDensN(idn.rhon,&grd,&param);



        // write E, B, rho to disk
        if (cycle%param.FieldOutputCycle==0){
            VTK_Write_Vectors(cycle, &grd,&field);
            VTK_Write_Scalars(cycle, &grd,ids,&idn);
        }

        eInterp += (cpuSecond() - iInterp); // stop timer for interpolation



    }  // end of one PIC cycle

    /// Release the resources
    // deallocate field
    grid_deallocate(&grd);
    field_deallocate(&grd,&field);
    // interp
    interp_dens_net_deallocate(&grd,&idn);

    // Deallocate interpolated densities and particles
    for (int is=0; is < param.ns; is++){
        interp_dens_species_deallocate(&grd,&ids[is]);
        particle_deallocate(&part[is]);
    }


    // stop timer
    double iElaps = cpuSecond() - iStart;

    // Print timing of simulation
    std::cout << std::endl;
    std::cout << "**************************************" << std::endl;
    std::cout << "   Tot. Simulation Time (s) = " << iElaps << std::endl;
    std::cout << "   Mover Time / Cycle   (s) = " << eMover/param.ncycles << std::endl;
    std::cout << "   Interp. Time / Cycle (s) = " << eInterp/param.ncycles  << std::endl;
    std::cout << "**************************************" << std::endl;

    // exit
    return 0;
}




Overwriting src/sputniPIC.cpp


In [196]:
%%writefile include/Particles.h
#ifndef PARTICLES_H
#define PARTICLES_H

#include <math.h>

#include "Alloc.h"
#include "Parameters.h"
#include "PrecisionTypes.h"
#include "Grid.h"
#include "EMfield.h"
#include "InterpDensSpecies.h"

struct particles {

    /** species ID: 0, 1, 2 , ... */
    int species_ID;

    /** maximum number of particles of this species on this domain. used for memory allocation */
    long npmax;
    /** number of particles of this species on this domain */
    long nop;

    /** Electron and ions have different number of iterations: ions moves slower than ions */
    int NiterMover;
    /** number of particle of subcycles in the mover */
    int n_sub_cycles;


    /** number of particles per cell */
    int npcel;
    /** number of particles per cell - X direction */
    int npcelx;
    /** number of particles per cell - Y direction */
    int npcely;
    /** number of particles per cell - Z direction */
    int npcelz;


    /** charge over mass ratio */
    FPpart qom;

    /* drift and thermal velocities for this species */
    FPpart u0, v0, w0;
    FPpart uth, vth, wth;

    /** particle arrays: 1D arrays[npmax] */
    FPpart* x; FPpart*  y; FPpart* z; FPpart* u; FPpart* v; FPpart* w;
    /** q must have precision of interpolated quantities: typically double. Not used in mover */
    FPinterp* q;



};

/** allocate particle arrays */
void particle_allocate(struct parameters*, struct particles*, int);

/** deallocate */
void particle_deallocate(struct particles*);

/** particle mover */
int mover_PC(struct particles*, struct EMfield*, struct grid*, struct parameters*);

/** gpu particle mover **/
int mover_PC_gpu(struct particles*, struct EMfield*, struct grid*, struct parameters*);

/** Interpolation Particle --> Grid: This is for species */
void interpP2G(struct particles*, struct interpDensSpecies*, struct grid*);

#endif


Overwriting include/Particles.h


In [192]:
%%writefile src/Particles.cu

#include "Particles.h"
#include "Alloc.h"
#include <cuda.h>
#include <cuda_runtime.h>

/** allocate particle arrays */
void particle_allocate(struct parameters* param, struct particles* part, int is)
{

    // set species ID
    part->species_ID = is;
    // number of particles
    part->nop = param->np[is];
    // maximum number of particles
    part->npmax = param->npMax[is];

    // choose a different number of mover iterations for ions and electrons
    if (param->qom[is] < 0){  //electrons
        part->NiterMover = param->NiterMover;
        part->n_sub_cycles = param->n_sub_cycles;
    } else {                  // ions: only one iteration
        part->NiterMover = 1;
        part->n_sub_cycles = 1;
    }

    // particles per cell
    part->npcelx = param->npcelx[is];
    part->npcely = param->npcely[is];
    part->npcelz = param->npcelz[is];
    part->npcel = part->npcelx*part->npcely*part->npcelz;

    // cast it to required precision
    part->qom = (FPpart) param->qom[is];

    long npmax = part->npmax;

    // initialize drift and thermal velocities
    // drift
    part->u0 = (FPpart) param->u0[is];
    part->v0 = (FPpart) param->v0[is];
    part->w0 = (FPpart) param->w0[is];
    // thermal
    part->uth = (FPpart) param->uth[is];
    part->vth = (FPpart) param->vth[is];
    part->wth = (FPpart) param->wth[is];


    //////////////////////////////
    /// ALLOCATION PARTICLE ARRAYS
    //////////////////////////////
    part->x = new FPpart[npmax];
    part->y = new FPpart[npmax];
    part->z = new FPpart[npmax];
    // allocate velocity
    part->u = new FPpart[npmax];
    part->v = new FPpart[npmax];
    part->w = new FPpart[npmax];
    // allocate charge = q * statistical weight
    part->q = new FPinterp[npmax];

}
/** deallocate */
void particle_deallocate(struct particles* part)
{
    // deallocate particle variables
    delete[] part->x;
    delete[] part->y;
    delete[] part->z;
    delete[] part->u;
    delete[] part->v;
    delete[] part->w;
    delete[] part->q;
}

__global__ void particleMoverKernel(struct particles* part, struct EMfield* field, struct grid* grd, struct parameters* param) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;

    // auxiliary variables
    FPpart dt_sub_cycling = (FPpart) param->dt/((double) part->n_sub_cycles);
    FPpart dto2 = .5*dt_sub_cycling, qomdt2 = part->qom*dto2/param->c;
    FPpart omdtsq, denom, ut, vt, wt, udotb;

    // local (to the particle) electric and magnetic field
    FPfield Exl=0.0, Eyl=0.0, Ezl=0.0, Bxl=0.0, Byl=0.0, Bzl=0.0;

    // intermediate particle position and velocity
    FPpart xptilde, yptilde, zptilde, uptilde, vptilde, wptilde;

    // interpolation densities
    int ix,iy,iz;
    FPfield weight[2][2][2];
    FPfield xi[2], eta[2], zeta[2];

    if (i < part->nop) {
      // move each particle with new fields --> each particle is handled independently so this can be done in parallel
      xptilde = part->x[i];
      yptilde = part->y[i];
      zptilde = part->z[i];
      // calculate the average velocity iteratively --> since it is iterative this cannot be parallelized
      for(int innter=0; innter < part->NiterMover; innter++){
          // interpolation G-->P
          ix = 2 +  int((part->x[i] - grd->xStart)*grd->invdx);
          iy = 2 +  int((part->y[i] - grd->yStart)*grd->invdy);
          iz = 2 +  int((part->z[i] - grd->zStart)*grd->invdz);

          // calculate weights
          xi[0]   = part->x[i] - grd->XN_flat[get_idx(ix - 1, iy, iz, grd->nyn, grd->nzn)]; //get_idx function from Alloc.h
          eta[0]  = part->y[i] - grd->YN_flat[get_idx(ix, iy - 1, iz, grd->nyn, grd->nzn)];
          zeta[0] = part->z[i] - grd->ZN_flat[get_idx(ix, iy, iz - 1, grd->nyn, grd->nzn)];
          xi[1]   = grd->XN_flat[get_idx(ix, iy, iz, grd->nyn, grd->nzn)] - part->x[i];
          eta[1]  = grd->YN_flat[get_idx(ix, iy, iz, grd->nyn, grd->nzn)] - part->y[i];
          zeta[1] = grd->ZN_flat[get_idx(ix, iy, iz, grd->nyn, grd->nzn)] - part->z[i];

          // set to zero local electric and magnetic field
          Exl=0.0, Eyl = 0.0, Ezl = 0.0, Bxl = 0.0, Byl = 0.0, Bzl = 0.0;

          // this loop and the one below can be merged
          // here we can flatten the weight array and do this in parallel as well
          for (int ii = 0; ii < 2; ii++)
              for (int jj = 0; jj < 2; jj++)
                  for (int kk = 0; kk < 2; kk++) {
                      weight[ii][jj][kk] = xi[ii] * eta[jj] * zeta[kk] * grd->invVOL;

                      Exl += weight[ii][jj][kk]*field->Ex_flat[get_idx(ix- ii, iy -jj, iz- kk, grd->nyn, grd->nzn)];
                      Eyl += weight[ii][jj][kk]*field->Ey_flat[get_idx(ix- ii, iy -jj, iz- kk, grd->nyn, grd->nzn)];
                      Ezl += weight[ii][jj][kk]*field->Ez_flat[get_idx(ix- ii, iy -jj, iz -kk, grd->nyn, grd->nzn)];
                      Bxl += weight[ii][jj][kk]*field->Bxn_flat[get_idx(ix- ii, iy -jj, iz -kk, grd->nyn, grd->nzn)];
                      Byl += weight[ii][jj][kk]*field->Byn_flat[get_idx(ix- ii, iy -jj, iz -kk, grd->nyn, grd->nzn)];
                      Bzl += weight[ii][jj][kk]*field->Bzn_flat[get_idx(ix- ii, iy -jj, iz -kk, grd->nyn, grd->nzn)];
                  }

          // end interpolation
          omdtsq = qomdt2*qomdt2*(Bxl*Bxl+Byl*Byl+Bzl*Bzl);
          denom = 1.0/(1.0 + omdtsq);
          // solve the position equation
          ut= part->u[i] + qomdt2*Exl;
          vt= part->v[i] + qomdt2*Eyl;
          wt= part->w[i] + qomdt2*Ezl;
          udotb = ut*Bxl + vt*Byl + wt*Bzl;
          // solve the velocity equation
          uptilde = (ut+qomdt2*(vt*Bzl -wt*Byl + qomdt2*udotb*Bxl))*denom;
          vptilde = (vt+qomdt2*(wt*Bxl -ut*Bzl + qomdt2*udotb*Byl))*denom;
          wptilde = (wt+qomdt2*(ut*Byl -vt*Bxl + qomdt2*udotb*Bzl))*denom;
          // update position
          part->x[i] = xptilde + uptilde*dto2;
          part->y[i] = yptilde + vptilde*dto2;
          part->z[i] = zptilde + wptilde*dto2;


      } // end of iteration
      // update the final position and velocity
      part->u[i]= 2.0*uptilde - part->u[i];
      part->v[i]= 2.0*vptilde - part->v[i];
      part->w[i]= 2.0*wptilde - part->w[i];
      part->x[i] = xptilde + uptilde*dt_sub_cycling;
      part->y[i] = yptilde + vptilde*dt_sub_cycling;
      part->z[i] = zptilde + wptilde*dt_sub_cycling;


      //////////
      //////////
      ////////// BC

      // X-DIRECTION: BC particles
      if (part->x[i] > grd->Lx){
          if (param->PERIODICX==true){ // PERIODIC
              part->x[i] = part->x[i] - grd->Lx;
          } else { // REFLECTING BC
              part->u[i] = -part->u[i];
              part->x[i] = 2*grd->Lx - part->x[i];
          }
      }

      if (part->x[i] < 0){
          if (param->PERIODICX==true){ // PERIODIC
              part->x[i] = part->x[i] + grd->Lx;
          } else { // REFLECTING BC
              part->u[i] = -part->u[i];
              part->x[i] = -part->x[i];
          }
      }


      // Y-DIRECTION: BC particles
      if (part->y[i] > grd->Ly){
          if (param->PERIODICY==true){ // PERIODIC
              part->y[i] = part->y[i] - grd->Ly;
          } else { // REFLECTING BC
              part->v[i] = -part->v[i];
              part->y[i] = 2*grd->Ly - part->y[i];
          }
      }

      if (part->y[i] < 0){
          if (param->PERIODICY==true){ // PERIODIC
              part->y[i] = part->y[i] + grd->Ly;
          } else { // REFLECTING BC
              part->v[i] = -part->v[i];
              part->y[i] = -part->y[i];
          }
      }

      // Z-DIRECTION: BC particles
      if (part->z[i] > grd->Lz){
          if (param->PERIODICZ==true){ // PERIODIC
              part->z[i] = part->z[i] - grd->Lz;
          } else { // REFLECTING BC
              part->w[i] = -part->w[i];
              part->z[i] = 2*grd->Lz - part->z[i];
          }
      }

      if (part->z[i] < 0){
          if (param->PERIODICZ==true){ // PERIODIC
              part->z[i] = part->z[i] + grd->Lz;
          } else { // REFLECTING BC
              part->w[i] = -part->w[i];
              part->z[i] = -part->z[i];
          }
      }
    }
}

void cuda_allocate_part(particles* h_part, particles* d_part, particles** d_part_ptr) {
    //allocate space in cuda
    cudaMalloc(&d_part->x, h_part->npmax * sizeof(FPpart));
    cudaMalloc(&d_part->y, h_part->npmax * sizeof(FPpart));
    cudaMalloc(&d_part->z, h_part->npmax * sizeof(FPpart));
    cudaMalloc(&d_part->u, h_part->npmax * sizeof(FPpart));
    cudaMalloc(&d_part->v, h_part->npmax * sizeof(FPpart));
    cudaMalloc(&d_part->w, h_part->npmax * sizeof(FPpart));
    cudaMalloc(d_part_ptr, sizeof(particles));

    // Copy the entire struct to the GPU
    cudaMemcpy(*d_part_ptr, h_part, sizeof(struct particles), cudaMemcpyHostToDevice);

    // Copy the content of each array in the struct to the GPU
    cudaMemcpy(d_part->x, h_part->x, h_part->npmax * sizeof(FPpart), cudaMemcpyHostToDevice);
    cudaMemcpy(d_part->y, h_part->y, h_part->npmax * sizeof(FPpart), cudaMemcpyHostToDevice);
    cudaMemcpy(d_part->z, h_part->z, h_part->npmax * sizeof(FPpart), cudaMemcpyHostToDevice);
    cudaMemcpy(d_part->u, h_part->u, h_part->npmax * sizeof(FPpart), cudaMemcpyHostToDevice);
    cudaMemcpy(d_part->v, h_part->v, h_part->npmax * sizeof(FPpart), cudaMemcpyHostToDevice);
    cudaMemcpy(d_part->w, h_part->w, h_part->npmax * sizeof(FPpart), cudaMemcpyHostToDevice);

    // bind the address on the correct field
    cudaMemcpy(&(*d_part_ptr)->x, &d_part->x, sizeof(FPpart*), cudaMemcpyHostToDevice);
    cudaMemcpy(&(*d_part_ptr)->y, &d_part->y, sizeof(FPpart*), cudaMemcpyHostToDevice);
    cudaMemcpy(&(*d_part_ptr)->z, &d_part->z, sizeof(FPpart*), cudaMemcpyHostToDevice);
    cudaMemcpy(&(*d_part_ptr)->u, &d_part->u, sizeof(FPpart*), cudaMemcpyHostToDevice);
    cudaMemcpy(&(*d_part_ptr)->v, &d_part->v, sizeof(FPpart*), cudaMemcpyHostToDevice);
    cudaMemcpy(&(*d_part_ptr)->w, &d_part->w, sizeof(FPpart*), cudaMemcpyHostToDevice);
}

// Copies result back to CPU and deallocates
void cuda_deallocate_part(struct particles* d_part, struct particles* h_part) {
    // move data to CPU
    cudaMemcpy(h_part->x, d_part->x, h_part->npmax * sizeof(FPpart), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_part->y, d_part->y, h_part->npmax * sizeof(FPpart), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_part->z, d_part->z, h_part->npmax * sizeof(FPpart), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_part->u, d_part->u, h_part->npmax * sizeof(FPpart), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_part->v, d_part->v, h_part->npmax * sizeof(FPpart), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_part->w, d_part->w, h_part->npmax * sizeof(FPpart), cudaMemcpyDeviceToHost);

    cudaFree(d_part->x);
    cudaFree(d_part->y);
    cudaFree(d_part->z);
    cudaFree(d_part->u);
    cudaFree(d_part->v);
    cudaFree(d_part->w);
}

//GPU version of particle mover
int mover_PC_gpu(struct particles* part, struct EMfield* field, struct grid* grd, struct parameters* param)
{
    // print species and subcycling
    std::cout << "***  GPU MOVER with SUBCYCLYING "<< param->n_sub_cycles << " - species " << part->species_ID << " ***" << std::endl;

    // define device variables
    struct particles* d_part_ptr;
    struct particles d_part;
    struct EMfield* d_field;
    struct grid* d_grd;
    struct parameters* d_param;

    // allocate and copy struct
    cudaMalloc(&d_field, sizeof(struct EMfield));
    cudaMalloc(&d_grd, sizeof(struct grid));
    cudaMalloc(&d_param, sizeof(struct parameters));

    cudaMemcpy(d_field, &field, sizeof(struct EMfield), cudaMemcpyHostToDevice);
    cudaMemcpy(d_grd, &grd, sizeof(struct grid), cudaMemcpyHostToDevice);
    cudaMemcpy(d_param, &param, sizeof(struct parameters), cudaMemcpyHostToDevice);

    //std:: cout << "allocating particle on device memory...\n";
    cuda_allocate_part(part, &d_part, &d_part_ptr);
    //std:: cout << "done\n";

    //std:: cout << "allocating field on device memory...\n";
    // field
    long field_size = grd->nxn * grd->nyn * grd->nzn;

    // allocate flattened arrays on device
    FPfield *d_Ex_flat, *d_Ey_flat, *d_Ez_flat, *d_Bxn_flat, *d_Byn_flat, *d_Bzn_flat;
    cudaMalloc(&d_Ex_flat, field_size * sizeof(FPfield));
    cudaMalloc(&d_Ey_flat, field_size * sizeof(FPfield));
    cudaMalloc(&d_Ez_flat, field_size * sizeof(FPfield));
    cudaMalloc(&d_Bxn_flat, field_size * sizeof(FPfield));
    cudaMalloc(&d_Byn_flat, field_size * sizeof(FPfield));
    cudaMalloc(&d_Bzn_flat, field_size * sizeof(FPfield));

    //copy content from host to device
    cudaMemcpy(d_Ex_flat, field->Ex_flat, field_size * sizeof(FPfield), cudaMemcpyHostToDevice);
    cudaMemcpy(d_Ey_flat, field->Ey_flat, field_size * sizeof(FPfield), cudaMemcpyHostToDevice);
    cudaMemcpy(d_Ez_flat, field->Ez_flat, field_size * sizeof(FPfield), cudaMemcpyHostToDevice);
    cudaMemcpy(d_Bxn_flat, field->Bxn_flat, field_size * sizeof(FPfield), cudaMemcpyHostToDevice);
    cudaMemcpy(d_Byn_flat, field->Byn_flat, field_size * sizeof(FPfield), cudaMemcpyHostToDevice);
    cudaMemcpy(d_Bzn_flat, field->Bzn_flat, field_size * sizeof(FPfield), cudaMemcpyHostToDevice);

    //bind addresses
    cudaMemcpy(&(d_field->Ex_flat), field->Ex_flat, sizeof(FPfield*), cudaMemcpyHostToDevice);
    cudaMemcpy(&(d_field->Ey_flat), field->Ey_flat, sizeof(FPfield*), cudaMemcpyHostToDevice);
    cudaMemcpy(&(d_field->Ez_flat), field->Ez_flat, sizeof(FPfield*), cudaMemcpyHostToDevice);
    cudaMemcpy(&(d_field->Bxn_flat), field->Bxn_flat, sizeof(FPfield*), cudaMemcpyHostToDevice);
    cudaMemcpy(&(d_field->Byn_flat), field->Byn_flat, sizeof(FPfield*), cudaMemcpyHostToDevice);
    cudaMemcpy(&(d_field->Bzn_flat), field->Bzn_flat, sizeof(FPfield*), cudaMemcpyHostToDevice);


    //std:: cout << "allocating grid on device memory...\n";

    // grid
    FPfield *d_XN_flat, *d_YN_flat, *d_ZN_flat; //copy of the arrays of the grid struct
    cudaMalloc(&d_XN_flat, field_size * sizeof(FPfield));
    cudaMalloc(&d_YN_flat, field_size * sizeof(FPfield));
    cudaMalloc(&d_ZN_flat, field_size * sizeof(FPfield));

    // copy content of flattened array
    cudaMemcpy(d_XN_flat, grd->XN_flat, field_size * sizeof(FPfield), cudaMemcpyHostToDevice);
    cudaMemcpy(d_YN_flat, grd->YN_flat, field_size * sizeof(FPfield), cudaMemcpyHostToDevice);
    cudaMemcpy(d_ZN_flat, grd->ZN_flat, field_size * sizeof(FPfield), cudaMemcpyHostToDevice);

    // bind the address on the correct field
    cudaMemcpy(&(d_grd)->XN_flat, &d_XN_flat, sizeof(FPfield*), cudaMemcpyHostToDevice);
    cudaMemcpy(&(d_grd)->YN_flat, &d_YN_flat, sizeof(FPfield*), cudaMemcpyHostToDevice);
    cudaMemcpy(&(d_grd)->ZN_flat, &d_ZN_flat, sizeof(FPfield*), cudaMemcpyHostToDevice);

    //std:: cout << "done\n";

    int TPB = 1024; //max numbers of thread on Tesla T4
    int numBlocks = (part->nop + TPB - 1) / TPB;

    // start subcycling
    for (int i_sub=0; i_sub <  part->n_sub_cycles; i_sub++){
        // call CUDA kernel
        particleMoverKernel<<<numBlocks, TPB>>>(d_part_ptr, d_field, d_grd, d_param);
        cudaDeviceSynchronize(); // Wait for the kernel to finish
    } // end of one particle


    //Copy the GPU memory back to the CPU
    cuda_deallocate_part(&d_part, part);
    cudaFree(d_part_ptr);


    cudaFree(d_Ex_flat);
    cudaFree(d_Ey_flat);
    cudaFree(d_Ez_flat);
    cudaFree(d_Bxn_flat);
    cudaFree(d_Byn_flat);
    cudaFree(d_Bzn_flat);
    cudaFree(d_field);

    cudaFree(d_XN_flat);
    cudaFree(d_YN_flat);
    cudaFree(d_ZN_flat);
    cudaFree(d_grd);

    cudaFree(d_param);

    return(0); // exit succcesfully
} // end of the mover


/** particle mover */
int mover_PC(struct particles* part, struct EMfield* field, struct grid* grd, struct parameters* param)
{
    // print species and subcycling
    std::cout << "***  MOVER with SUBCYCLYING "<< param->n_sub_cycles << " - species " << part->species_ID << " ***" << std::endl;

    // auxiliary variables
    FPpart dt_sub_cycling = (FPpart) param->dt/((double) part->n_sub_cycles);
    FPpart dto2 = .5*dt_sub_cycling, qomdt2 = part->qom*dto2/param->c;
    FPpart omdtsq, denom, ut, vt, wt, udotb;

    // local (to the particle) electric and magnetic field
    FPfield Exl=0.0, Eyl=0.0, Ezl=0.0, Bxl=0.0, Byl=0.0, Bzl=0.0;

    // interpolation densities
    int ix,iy,iz;
    FPfield weight[2][2][2];
    FPfield xi[2], eta[2], zeta[2];

    // intermediate particle position and velocity
    FPpart xptilde, yptilde, zptilde, uptilde, vptilde, wptilde;

    // start subcycling
    for (int i_sub=0; i_sub <  part->n_sub_cycles; i_sub++){
        // move each particle with new fields
        for (int i=0; i <  part->nop; i++){
            xptilde = part->x[i];
            yptilde = part->y[i];
            zptilde = part->z[i];
            // calculate the average velocity iteratively
            for(int innter=0; innter < part->NiterMover; innter++){
                // interpolation G-->P
                ix = 2 +  int((part->x[i] - grd->xStart)*grd->invdx);
                iy = 2 +  int((part->y[i] - grd->yStart)*grd->invdy);
                iz = 2 +  int((part->z[i] - grd->zStart)*grd->invdz);

                // calculate weights
                xi[0]   = part->x[i] - grd->XN[ix - 1][iy][iz];
                eta[0]  = part->y[i] - grd->YN[ix][iy - 1][iz];
                zeta[0] = part->z[i] - grd->ZN[ix][iy][iz - 1];
                xi[1]   = grd->XN[ix][iy][iz] - part->x[i];
                eta[1]  = grd->YN[ix][iy][iz] - part->y[i];
                zeta[1] = grd->ZN[ix][iy][iz] - part->z[i];
                for (int ii = 0; ii < 2; ii++)
                    for (int jj = 0; jj < 2; jj++)
                        for (int kk = 0; kk < 2; kk++)
                            weight[ii][jj][kk] = xi[ii] * eta[jj] * zeta[kk] * grd->invVOL;

                // set to zero local electric and magnetic field
                Exl=0.0, Eyl = 0.0, Ezl = 0.0, Bxl = 0.0, Byl = 0.0, Bzl = 0.0;

                for (int ii=0; ii < 2; ii++)
                    for (int jj=0; jj < 2; jj++)
                        for(int kk=0; kk < 2; kk++){
                            Exl += weight[ii][jj][kk]*field->Ex[ix- ii][iy -jj][iz- kk ];
                            Eyl += weight[ii][jj][kk]*field->Ey[ix- ii][iy -jj][iz- kk ];
                            Ezl += weight[ii][jj][kk]*field->Ez[ix- ii][iy -jj][iz -kk ];
                            Bxl += weight[ii][jj][kk]*field->Bxn[ix- ii][iy -jj][iz -kk ];
                            Byl += weight[ii][jj][kk]*field->Byn[ix- ii][iy -jj][iz -kk ];
                            Bzl += weight[ii][jj][kk]*field->Bzn[ix- ii][iy -jj][iz -kk ];
                        }

                // end interpolation
                omdtsq = qomdt2*qomdt2*(Bxl*Bxl+Byl*Byl+Bzl*Bzl);
                denom = 1.0/(1.0 + omdtsq);
                // solve the position equation
                ut= part->u[i] + qomdt2*Exl;
                vt= part->v[i] + qomdt2*Eyl;
                wt= part->w[i] + qomdt2*Ezl;
                udotb = ut*Bxl + vt*Byl + wt*Bzl;
                // solve the velocity equation
                uptilde = (ut+qomdt2*(vt*Bzl -wt*Byl + qomdt2*udotb*Bxl))*denom;
                vptilde = (vt+qomdt2*(wt*Bxl -ut*Bzl + qomdt2*udotb*Byl))*denom;
                wptilde = (wt+qomdt2*(ut*Byl -vt*Bxl + qomdt2*udotb*Bzl))*denom;
                // update position
                part->x[i] = xptilde + uptilde*dto2;
                part->y[i] = yptilde + vptilde*dto2;
                part->z[i] = zptilde + wptilde*dto2;


            } // end of iteration
            // update the final position and velocity
            part->u[i]= 2.0*uptilde - part->u[i];
            part->v[i]= 2.0*vptilde - part->v[i];
            part->w[i]= 2.0*wptilde - part->w[i];
            part->x[i] = xptilde + uptilde*dt_sub_cycling;
            part->y[i] = yptilde + vptilde*dt_sub_cycling;
            part->z[i] = zptilde + wptilde*dt_sub_cycling;


            //////////
            //////////
            ////////// BC

            // X-DIRECTION: BC particles
            if (part->x[i] > grd->Lx){
                if (param->PERIODICX==true){ // PERIODIC
                    part->x[i] = part->x[i] - grd->Lx;
                } else { // REFLECTING BC
                    part->u[i] = -part->u[i];
                    part->x[i] = 2*grd->Lx - part->x[i];
                }
            }

            if (part->x[i] < 0){
                if (param->PERIODICX==true){ // PERIODIC
                   part->x[i] = part->x[i] + grd->Lx;
                } else { // REFLECTING BC
                    part->u[i] = -part->u[i];
                    part->x[i] = -part->x[i];
                }
            }


            // Y-DIRECTION: BC particles
            if (part->y[i] > grd->Ly){
                if (param->PERIODICY==true){ // PERIODIC
                    part->y[i] = part->y[i] - grd->Ly;
                } else { // REFLECTING BC
                    part->v[i] = -part->v[i];
                    part->y[i] = 2*grd->Ly - part->y[i];
                }
            }

            if (part->y[i] < 0){
                if (param->PERIODICY==true){ // PERIODIC
                    part->y[i] = part->y[i] + grd->Ly;
                } else { // REFLECTING BC
                    part->v[i] = -part->v[i];
                    part->y[i] = -part->y[i];
                }
            }

            // Z-DIRECTION: BC particles
            if (part->z[i] > grd->Lz){
                if (param->PERIODICZ==true){ // PERIODIC
                    part->z[i] = part->z[i] - grd->Lz;
                } else { // REFLECTING BC
                    part->w[i] = -part->w[i];
                    part->z[i] = 2*grd->Lz - part->z[i];
                }
            }

            if (part->z[i] < 0){
                if (param->PERIODICZ==true){ // PERIODIC
                    part->z[i] = part->z[i] + grd->Lz;
                } else { // REFLECTING BC
                    part->w[i] = -part->w[i];
                    part->z[i] = -part->z[i];
                }
            }



        }  // end of subcycling
    } // end of one particle

    return(0); // exit succcesfully
} // end of the mover



/** Interpolation Particle --> Grid: This is for species */
void interpP2G(struct particles* part, struct interpDensSpecies* ids, struct grid* grd)
{

    // arrays needed for interpolation
    FPpart weight[2][2][2];
    FPpart temp[2][2][2];
    FPpart xi[2], eta[2], zeta[2];

    // index of the cell
    int ix, iy, iz;


    for (register long long i = 0; i < part->nop; i++) {

        // determine cell: can we change to int()? is it faster?
        ix = 2 + int (floor((part->x[i] - grd->xStart) * grd->invdx));
        iy = 2 + int (floor((part->y[i] - grd->yStart) * grd->invdy));
        iz = 2 + int (floor((part->z[i] - grd->zStart) * grd->invdz));

        // distances from node
        xi[0]   = part->x[i] - grd->XN[ix - 1][iy][iz];
        eta[0]  = part->y[i] - grd->YN[ix][iy - 1][iz];
        zeta[0] = part->z[i] - grd->ZN[ix][iy][iz - 1];
        xi[1]   = grd->XN[ix][iy][iz] - part->x[i];
        eta[1]  = grd->YN[ix][iy][iz] - part->y[i];
        zeta[1] = grd->ZN[ix][iy][iz] - part->z[i];

        // calculate the weights for different nodes
        for (int ii = 0; ii < 2; ii++)
            for (int jj = 0; jj < 2; jj++)
                for (int kk = 0; kk < 2; kk++)
                    weight[ii][jj][kk] = part->q[i] * xi[ii] * eta[jj] * zeta[kk] * grd->invVOL;

        //////////////////////////
        // add charge density
        for (int ii = 0; ii < 2; ii++)
            for (int jj = 0; jj < 2; jj++)
                for (int kk = 0; kk < 2; kk++)
                    ids->rhon[ix - ii][iy - jj][iz - kk] += weight[ii][jj][kk] * grd->invVOL;


        ////////////////////////////
        // add current density - Jx
        for (int ii = 0; ii < 2; ii++)
            for (int jj = 0; jj < 2; jj++)
                for (int kk = 0; kk < 2; kk++)
                    temp[ii][jj][kk] = part->u[i] * weight[ii][jj][kk];

        for (int ii = 0; ii < 2; ii++)
            for (int jj = 0; jj < 2; jj++)
                for (int kk = 0; kk < 2; kk++)
                    ids->Jx[ix - ii][iy - jj][iz - kk] += temp[ii][jj][kk] * grd->invVOL;


        ////////////////////////////
        // add current density - Jy
        for (int ii = 0; ii < 2; ii++)
            for (int jj = 0; jj < 2; jj++)
                for (int kk = 0; kk < 2; kk++)
                    temp[ii][jj][kk] = part->v[i] * weight[ii][jj][kk];
        for (int ii = 0; ii < 2; ii++)
            for (int jj = 0; jj < 2; jj++)
                for (int kk = 0; kk < 2; kk++)
                    ids->Jy[ix - ii][iy - jj][iz - kk] += temp[ii][jj][kk] * grd->invVOL;



        ////////////////////////////
        // add current density - Jz
        for (int ii = 0; ii < 2; ii++)
            for (int jj = 0; jj < 2; jj++)
                for (int kk = 0; kk < 2; kk++)
                    temp[ii][jj][kk] = part->w[i] * weight[ii][jj][kk];
        for (int ii = 0; ii < 2; ii++)
            for (int jj = 0; jj < 2; jj++)
                for (int kk = 0; kk < 2; kk++)
                    ids->Jz[ix - ii][iy - jj][iz - kk] += temp[ii][jj][kk] * grd->invVOL;


        ////////////////////////////
        // add pressure pxx
        for (int ii = 0; ii < 2; ii++)
            for (int jj = 0; jj < 2; jj++)
                for (int kk = 0; kk < 2; kk++)
                    temp[ii][jj][kk] = part->u[i] * part->u[i] * weight[ii][jj][kk];
        for (int ii = 0; ii < 2; ii++)
            for (int jj = 0; jj < 2; jj++)
                for (int kk = 0; kk < 2; kk++)
                    ids->pxx[ix - ii][iy - jj][iz - kk] += temp[ii][jj][kk] * grd->invVOL;


        ////////////////////////////
        // add pressure pxy
        for (int ii = 0; ii < 2; ii++)
            for (int jj = 0; jj < 2; jj++)
                for (int kk = 0; kk < 2; kk++)
                    temp[ii][jj][kk] = part->u[i] * part->v[i] * weight[ii][jj][kk];
        for (int ii = 0; ii < 2; ii++)
            for (int jj = 0; jj < 2; jj++)
                for (int kk = 0; kk < 2; kk++)
                    ids->pxy[ix - ii][iy - jj][iz - kk] += temp[ii][jj][kk] * grd->invVOL;



        /////////////////////////////
        // add pressure pxz
        for (int ii = 0; ii < 2; ii++)
            for (int jj = 0; jj < 2; jj++)
                for (int kk = 0; kk < 2; kk++)
                    temp[ii][jj][kk] = part->u[i] * part->w[i] * weight[ii][jj][kk];
        for (int ii = 0; ii < 2; ii++)
            for (int jj = 0; jj < 2; jj++)
                for (int kk = 0; kk < 2; kk++)
                    ids->pxz[ix - ii][iy - jj][iz - kk] += temp[ii][jj][kk] * grd->invVOL;


        /////////////////////////////
        // add pressure pyy
        for (int ii = 0; ii < 2; ii++)
            for (int jj = 0; jj < 2; jj++)
                for (int kk = 0; kk < 2; kk++)
                    temp[ii][jj][kk] = part->v[i] * part->v[i] * weight[ii][jj][kk];
        for (int ii = 0; ii < 2; ii++)
            for (int jj = 0; jj < 2; jj++)
                for (int kk = 0; kk < 2; kk++)
                    ids->pyy[ix - ii][iy - jj][iz - kk] += temp[ii][jj][kk] * grd->invVOL;


        /////////////////////////////
        // add pressure pyz
        for (int ii = 0; ii < 2; ii++)
            for (int jj = 0; jj < 2; jj++)
                for (int kk = 0; kk < 2; kk++)
                    temp[ii][jj][kk] = part->v[i] * part->w[i] * weight[ii][jj][kk];
        for (int ii = 0; ii < 2; ii++)
            for (int jj = 0; jj < 2; jj++)
                for (int kk = 0; kk < 2; kk++)
                    ids->pyz[ix - ii][iy - jj][iz - kk] += temp[ii][jj][kk] * grd->invVOL;


        /////////////////////////////
        // add pressure pzz
        for (int ii = 0; ii < 2; ii++)
            for (int jj = 0; jj < 2; jj++)
                for (int kk = 0; kk < 2; kk++)
                    temp[ii][jj][kk] = part->w[i] * part->w[i] * weight[ii][jj][kk];
        for (int ii=0; ii < 2; ii++)
            for (int jj=0; jj < 2; jj++)
                for(int kk=0; kk < 2; kk++)
                    ids->pzz[ix -ii][iy -jj][iz - kk] += temp[ii][jj][kk] * grd->invVOL;

    }

}


Overwriting src/Particles.cu


In [193]:
%rm -rf bin
%ls

data/  include/  inputfiles/  Makefile  Particles.cu  src/


In [194]:
! make

mkdir -p ./bin
nvcc -I./include -arch=sm_75 -std=c++11 -O3 -g -Xcompiler -Wall --compiler-bindir=g++ src/Particles.cu -c -o src/Particles.o
src/Particles.cu: In function ‘int mover_PC(particles*, EMfield*, grid*, parameters*)’:
src/Particles.cu:471:38: warning: ‘wptilde’ may be used uninitialized in this function []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wmaybe-uninitialized-Wmaybe-uninitialized]8;;]
  471 |             part->z[i] = zptilde + wptilde*dt_sub_cycling;
      |                             ~~~~~~~~~^~~~~~~~~~~~~~~~~
src/Particles.cu:467:28: warning: ‘vptilde’ may be used uninitialized in this function []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wmaybe-uninitialized-Wmaybe-uninitialized]8;;]
  467 |             part->v[i]= 2.0*vptilde - part->v[i];
      |                            ^~~~~~~
src/Particles.cu:466:28: warning: ‘uptilde’ may be used uninitialized in this function []8;;https://gcc.gnu.org/onlinedocs/gcc/War

In [190]:
# run the code with nsight
! /usr/local/cuda-11/bin/nv-nsight-cu-cli ./bin/sputniPIC.out inputfiles/GEM_2D.inp


-------------------------
sputniPIC Sim. Parameters
-------------------------
Number of species    = 4
Number of particles of species 0 = 4096000	 (MAX = 4096000)  QOM = -64
Number of particles of species 1 = 4096000	 (MAX = 4096000)  QOM = 1
Number of particles of species 2 = 4096000	 (MAX = 4096000)  QOM = -64
Number of particles of species 3 = 4096000	 (MAX = 4096000)  QOM = 1
x-Length                 = 40
y-Length                 = 20
z-Length                 = 1
Number of cells (x)      = 256
Number of cells (y)      = 128
Number of cells (z)      = 1
Time step                = 0.25
Number of cycles         = 10
Results saved in: data
*************************************************
**  Initialize GEM Challenge with Pertubation  **
*************************************************
** B0x = 0.0195
** B0y = 0
** B0z = 0
** Delta (current sheet thickness) = 0.5
** rho species 0 = 1 CURRENT SHEET 
** rho species 1 = 1 CURRENT SHEET 
** rho species 2 = 0.02 BACKGROUND 
** rho species

In [195]:
! ./bin/sputniPIC.out inputfiles/GEM_2D.inp


-------------------------
sputniPIC Sim. Parameters
-------------------------
Number of species    = 4
Number of particles of species 0 = 4096000	 (MAX = 4096000)  QOM = -64
Number of particles of species 1 = 4096000	 (MAX = 4096000)  QOM = 1
Number of particles of species 2 = 4096000	 (MAX = 4096000)  QOM = -64
Number of particles of species 3 = 4096000	 (MAX = 4096000)  QOM = 1
x-Length                 = 40
y-Length                 = 20
z-Length                 = 1
Number of cells (x)      = 256
Number of cells (y)      = 128
Number of cells (z)      = 1
Time step                = 0.25
Number of cycles         = 10
Results saved in: data
*************************************************
**  Initialize GEM Challenge with Pertubation  **
*************************************************
** B0x = 0.0195
** B0y = 0
** B0z = 0
** Delta (current sheet thickness) = 0.5
** rho species 0 = 1 CURRENT SHEET 
** rho species 1 = 1 CURRENT SHEET 
** rho species 2 = 0.02 BACKGROUND 
** rho species